# Delta Table Optimization and Benchmarking
This notebook demonstrates how to optimize Delta tables in Databricks using partitioning and ZORDER, and benchmarks query performance before and after optimization.

In [0]:
%sql
-- set the working catalog and schema
use catalog workspace;
use schema default;

In [0]:
# Task 1 : Analyze the query execution plan for brand
# Analyze the query execution plan for brand
# Note that Databricks has Spark process the query before any optimization
sql = '''SELECT * FROM ecommerce_bronze_delta WHERE brand IS NOT NULL '''
display(spark.sql(sql).explain(True))

== Parsed Logical Plan ==
'Project [*]
+- 'Filter isnotnull('brand)
   +- 'UnresolvedRelation [ecommerce_bronze_delta], [], false

== Analyzed Logical Plan ==
event_time: string, event_type: string, product_id: string, category_id: string, category_code: string, brand: string, price: string, user_id: string, user_session: string
Project [event_time#13321, event_type#13322, product_id#13323, category_id#13324, category_code#13325, brand#13326, price#13327, user_id#13328, user_session#13329]
+- Filter isnotnull(brand#13326)
   +- SubqueryAlias workspace.default.ecommerce_bronze_delta
      +- Relation workspace.default.ecommerce_bronze_delta[event_time#13321,event_type#13322,product_id#13323,category_id#13324,category_code#13325,brand#13326,price#13327,user_id#13328,user_session#13329] parquet

== Optimized Logical Plan ==
Filter isnotnull(brand#13326)
+- Relation workspace.default.ecommerce_bronze_delta[event_time#13321,event_type#13322,product_id#13323,category_id#13324,category_code#1

In [0]:
# Task 2 : Partiion large tables
# Create a new optimized delta table partitioned by event_type
# Partitioning helps speed up queries that filter on these columns
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_ecommerce_events_event_type_part
USING DELTA
PARTITIONED BY (event_type)
AS
SELECT * 
FROM ecommerce_bronze_delta
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
-- Task 2 : Partiion large tables
-- show the partitioned table
show partitions silver_ecommerce_events_event_type_part

event_type
view
purchase
cart


In [0]:
%sql
-- Task 3 : apply ZORDER BY to the partitioned table
optimize silver_ecommerce_events_event_type_part ZORDER BY (product_id);

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 2, List(minCubeSize(107374182400), List(11, 660849625), List(2, 45898973), 1, List(0, 0), 0, null), null, 0, 0, 13, 13, false, 0, 0, 1768752930161, 1768752931291, 8, 0, null, List(0, 0), null, 9, 9, 0, 0, null)"


In [0]:
%sql
--Task4: Benchmark Improvements
-- BeforeOptimization
SELECT *
FROM ecommerce_bronze_delta
LIMIT 25

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-11 17:14:26 UTC,view,1004739,2053013555631882655,electronics.smartphone,xiaomi,188.02,507682847,dfc44355-6b7b-4bcd-8feb-89c022ed254d
2019-10-11 17:14:27 UTC,view,26300182,2053013563424899933,null,trollbeads,194.08,541632398,ca1496c2-e2af-4a38-bb85-adf72e52d31f
2019-10-11 17:14:27 UTC,view,1005003,2053013555631882655,electronics.smartphone,huawei,252.23,514447368,dec3ce89-b93e-4379-9d2d-7521e4304f3d
2019-10-11 17:14:27 UTC,view,22400122,2053013554474254687,electronics.audio.microphone,trust,110.94,519285494,81c776b3-fd06-449e-a982-108be660da2b
2019-10-11 17:14:27 UTC,view,15100227,2053013557024391671,null,null,731.04,549462175,d244b901-5a38-41f3-a184-719eeb972d6a
2019-10-11 17:14:27 UTC,view,49100007,2127425375913902544,null,laston,132.46,513544737,f9bd49a3-e637-4ccc-aff3-2cd6e64bb102
2019-10-11 17:14:27 UTC,view,4300382,2053013552385491165,null,xiaomi,118.15,541793923,92194a0a-0d43-4a6d-b86e-4ff373c46c6a
2019-10-11 17:14:27 UTC,view,28706889,2053013565127787455,apparel.shoes,respect,31.92,539240040,7ecae67c-a407-4b85-8e47-7a45f64d5bfe
2019-10-11 17:14:27 UTC,view,12703300,2053013553559896355,null,cordiant,64.35,513168503,a6c5a818-5e02-4243-b27b-f200e26c96cb
2019-10-11 17:14:27 UTC,view,32400557,2053013566562238479,null,freedom,19.82,554050042,544f56a6-51e7-40af-ad82-4346b042b2d4


In [0]:
%sql
--Task4: Benchmark Improvements
-- AfterOptimization
SELECT *
FROM silver_ecommerce_events_event_type_part
LIMIT 25

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-26 07:55:55 UTC,view,1004858,2053013555631882655,electronics.smartphone,samsung,131.53,522850155,40738b0d-76ac-4e20-8020-045709479161
2019-10-26 07:55:55 UTC,view,1004864,2053013555631882655,electronics.smartphone,honor,115.81,515027636,4933db9d-0ab5-4c65-b125-a3874e191da5
2019-10-26 07:55:56 UTC,view,1004767,2053013555631882655,electronics.smartphone,samsung,245.28,550404843,12d245f5-dc44-4416-8ca0-9a84a83e3029
2019-10-26 07:55:57 UTC,view,1004768,2053013555631882655,electronics.smartphone,samsung,245.26,557311182,d6de2f4d-5d70-4451-8e3c-3285a90663b7
2019-10-26 07:55:57 UTC,view,1004838,2053013555631882655,electronics.smartphone,oppo,141.29,564297430,0794f8b8-b9ff-45ad-82ab-c44f51a65595
2019-10-26 07:55:59 UTC,view,1004992,2053013555631882655,electronics.smartphone,oppo,231.64,512430659,30ea7c26-e402-449c-8fab-de33c6fe7b6e
2019-10-26 07:55:59 UTC,view,1004766,2053013555631882655,electronics.smartphone,samsung,244.32,563561200,ef1cd29b-e947-7877-c2d9-b0946db76598
2019-10-26 07:55:59 UTC,view,1004741,2053013555631882655,electronics.smartphone,xiaomi,189.97,540014734,6f4aee09-9db0-47a0-b18d-dc189dec1f30
2019-10-26 07:55:59 UTC,view,1004870,2053013555631882655,electronics.smartphone,samsung,281.11,547109676,23d19ab2-66d1-4b05-9e9f-c61af66e01df
2019-10-26 07:55:59 UTC,view,1004836,2053013555631882655,electronics.smartphone,samsung,223.76,533886616,5f285e5d-dca3-444c-bc5e-e728b2aa5621
